# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):




architecture screentshot:

![](20251120024008.png)

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

In [1]:
# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

training_data = list("""
# Transformers

this note book is here to help me refresh some of my understanding of the basic transformers architecture

we want to implement the encoder part of the architecture in [attention is all you need paper](https://arxiv.org/pdf/1706.03762):

My goal with be to go through one pass of transformer layer for a data, and try to explain each layer, finally I will convert this jupyter notebook to a python code and train it on a simple dataset

# I want this note book to be very simple so I will make the data very simple, i.e use whatever I have written till now as training data

""")

In [2]:
# I don't want to get too deep into tokenization for this notebook so I am just going to instead use all the unique characters
# present in the training data as distinct tokens
vocabulary_list = list[str](set(training_data))

In [3]:
print(vocabulary_list[:5])
print(len(vocabulary_list))

[')', 't', 'o', 'r', 'i']
44


In [4]:
# let's create training and testing data
# training and testing data for next token prediction would look something like

# the way the transformer works is that for a single example sentence it trains the model for multiple token prediction
print(training_data[:9])

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f']


In [5]:
# here if x is
training_data[:8]

['\n', '#', ' ', 'T', 'r', 'a', 'n', 's']

In [6]:
# then y would be
training_data[1:9]

['#', ' ', 'T', 'r', 'a', 'n', 's', 'f']

In [7]:
# ok before we make create training data we need to convert our tokens to a unique index to do that I will do
token_to_index = {c:i for i,c in enumerate(vocabulary_list)}
index_to_token = {i:c for i,c in enumerate(vocabulary_list)}

In [8]:
# now we let's convert our training data to a torch tensor
import torch

training_data_tensor = torch.tensor([token_to_index[c] for c in training_data], dtype=torch.long)

In [9]:
print(training_data_tensor[:10])
print([index_to_token[ix.item()] for ix in training_data_tensor[:10]])

tensor([37, 32, 41, 18,  3, 31, 43, 36, 42,  2])
['\n', '#', ' ', 'T', 'r', 'a', 'n', 's', 'f', 'o']


In [10]:
# now let's create training and testing set
block_size = 8
x = torch.stack([training_data_tensor[ix:ix+block_size] for ix in range(len(training_data_tensor)-block_size)] )
# max ix len(training_data_tensor)-block_size - 1
# so ix + block_size = len(training_data_tensor) - 1
# so final example won't include last character
y = torch.stack([training_data_tensor[ix:ix+block_size]for ix in range(1,len(training_data_tensor)-block_size+1)]) 



In [11]:
print("x training data")
print(x[:5])
print("y training data")
print(y[:5])

x training data
tensor([[37, 32, 41, 18,  3, 31, 43, 36],
        [32, 41, 18,  3, 31, 43, 36, 42],
        [41, 18,  3, 31, 43, 36, 42,  2],
        [18,  3, 31, 43, 36, 42,  2,  3],
        [ 3, 31, 43, 36, 42,  2,  3, 38]])
y training data
tensor([[32, 41, 18,  3, 31, 43, 36, 42],
        [41, 18,  3, 31, 43, 36, 42,  2],
        [18,  3, 31, 43, 36, 42,  2,  3],
        [ 3, 31, 43, 36, 42,  2,  3, 38],
        [31, 43, 36, 42,  2,  3, 38, 33]])


# Embedding Table

![](20251121001141.png)


This is a look up table between the vocabulary index and n dimensional vector,
during the training of transformer model this vectors also gets trained, i.e where these vectors point to gets updated,
based on the similarity between these vectors, if let's say I have 2 tokens "dog" and "pooch", during the start of training process
they might point in very different directions, but after the training both would point to pretty much same place

### Question?:

1. What is so special about the training process that transforms these vectors from pointing in random ass direction, to actually have some meaning
    * for now I am gonna assume that the answer is that the transformer architecture expects and assumes these vectors to be what I have described
    * and based on this assumption, the subsequent layers performs its operation, so optimizing the loss leads to these embedding vector looking more like actual high dimensional representation of the words 

In [12]:
from torch import nn

EMBEDDING_DIMENSION = 8
VOCAB_SIZE = len(vocabulary_list)

embeddings_table = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION)

In [13]:
# some experimentation on how embeddings table work,
print(embeddings_table(torch.tensor([[0,1,2,3]], dtype=torch.long)))
# it goes to each item in tensor and assumes each item is a index converts it to its corresponding embedding vector

tensor([[[-0.4503,  0.4298, -2.4783, -1.4611, -0.4113, -0.6076,  1.1511,
           0.1232],
         [ 1.6250,  0.0502, -1.2053, -0.0711, -1.2139, -0.0619,  0.4029,
           0.2897],
         [ 0.6079,  2.0361, -0.0104,  1.2162,  0.4097,  0.1147,  0.8930,
          -0.3359],
         [-0.7860,  1.2104,  0.4375,  2.3045, -1.2115, -0.0026,  0.3084,
           1.7509]]], grad_fn=<EmbeddingBackward0>)


I want to do a very simple forward pass so I am gonna create my forward pass batch now

In [14]:
x_batch = x[:5]
y_batch = y[:5]

A question lingers, what does this (shifted right) mean:

![](20251121002201.png)

this just means that our input is shifted from the target output

In [15]:
x_embeddings = embeddings_table(x_batch)

In [16]:
# just one example
x_embeddings[:1]

tensor([[[-0.4722,  0.0425, -1.3180, -0.3957, -0.0849,  0.8154, -1.2175,
           0.8029],
         [ 2.4556, -0.6168,  0.1083, -0.3513,  0.5772,  0.9587,  0.2648,
           0.1310],
         [-1.9843,  0.7081,  0.7081,  0.2608, -1.0187, -0.0744, -0.0911,
           0.2809],
         [-1.1661,  0.8945,  0.7230,  0.8823,  2.0409,  0.7549, -0.1857,
          -0.1812],
         [-0.7860,  1.2104,  0.4375,  2.3045, -1.2115, -0.0026,  0.3084,
           1.7509],
         [ 0.4898,  0.3357, -0.4497,  0.4210, -0.6362,  0.2130, -0.8093,
          -0.0829],
         [ 1.2873,  1.5426,  0.0798,  0.1818, -0.9855, -1.0565,  0.7111,
          -0.1300],
         [-2.0003,  0.5513,  1.1447,  0.6162,  0.7095, -1.0166,  0.1581,
          -0.5663]]], grad_fn=<SliceBackward0>)

# Positional Encoding

![](20251121135418.png)


From my past understanding this is sort of values with varies with the position of the token in the sequence to encode the information about the position of the token in the sequence

so for each position there will be a vector associated to it, which will get added to the original embedding vector at that position

### Questions?:
1. Why Add these vectors to the original embedding vector? Can it not be appended or create some other type of encoding create a new channel perhaps like we do for CNNs
    - Ans: The Idea behind adding these is how we treat embedding vectors, you can think of embedding vector as the original absolute meaning of a token, now depending on whether it appears at the beggining of a sentence or end of a sentence it's meaning might differ, i.e its embedding vector might change its position, that change is capture by the addition of this positional embeddding vector
2. Why do these needs to be a vector all together can these not be like a single number which gets added?
    - Ans: well a vector is a more generalized version of a single number, if single number is the right approach then expectation is that the network would train the embedings to become a single number

## Sinusoidal Encoding

![](20251121140214.png)

here d_model is the dimension of the embedding

In the original Paper they used a fix positional sinusoidal encoding, they mentioned the performance for both learned and not learned were identical, they wanted to experiment with sinusoidal encoding, because they wanted to test the model beyond the trained context length

# Question?:
1. But why sinusoidal encoding

In [17]:
positional_embedding_table = nn.Embedding(block_size, EMBEDDING_DIMENSION)

In [18]:
x_pos_embeddings = positional_embedding_table(torch.arange(x_embeddings.shape[1])) # C, E

x_embeddings # B, C, E

x_embeddings_total = x_embeddings + x_pos_embeddings # B, C, E + C, E -> pytorch checks the shape starting from right and if there is an extra dimension it creates a new dimention and copuies the same thing over, like C, E -> (1, C, E) -> (B, C, E)

# Self Attention Layer

![](20251123232032.png)

I will start of by explaining what this layer does in a high level, then I will dig deep into how it does this, initially I will go over a single head self attention, 
then understand myself and explain why multi head self attention

this is the 3b1b interpretation of this layer on a high level, which I found to be the most elegant

## The Explanation
This layer as a whole tells us how should the original embedding vector be modified, so that it's meaning is enriched with the context of the surrounding tokens, for example take the sentence:

" That blue aeroplane is very dangerous "

in this example initially "aeroplane"'s embedding vector would straight up point to the absolute aeroplane,
then attention layer outputs a result, that result when added to the original embedding vector, nudges the aeroplane's vector in a direction closer to blue and dangerous

that is on high level what this layer does, now going into the detail let's start by the equation

![](20251123233808.png)

the above represent the equation describing the self attention mechanism, for the purpose of this excercise we will focus on masked self attention

here Q, K, V are all matrices

Q being the query matrix, K Key matrix and V value matrix

let me do one thing and form this forumla in our on going example and then explain

In [19]:
d_k = d_q = 4

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, EMBEDDING_DIMENSION, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension


In [20]:
W_q.weight.shape

torch.Size([4, 8])

In [21]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [22]:
W_q.weight@x_embeddings_total[0][0]

tensor([ 1.7130, -0.7751,  1.1649, -0.1630], grad_fn=<MvBackward0>)

In [23]:
W_q(x_embeddings_total)

tensor([[[ 1.7130, -0.7751,  1.1649, -0.1630],
         [-0.6965, -0.9022,  0.4943, -0.9418],
         [ 1.3131, -0.2510,  1.5398,  0.0729],
         [ 0.5665, -0.1373,  0.9703, -0.2949],
         [-0.1282,  0.3207,  0.5693,  0.1479],
         [-0.3674, -0.7386,  0.3818, -0.7358],
         [-0.5556, -0.5934, -0.2064, -0.1368],
         [ 0.6535,  0.0146,  0.4936,  0.1099]],

        [[ 0.5938, -0.5721, -0.5111, -0.2725],
         [-0.0194, -0.5456,  2.3623,  0.1166],
         [ 1.6750, -0.4455,  1.5687, -0.5050],
         [-0.1081,  0.4488,  1.1117,  0.1698],
         [-0.3539, -0.6767,  0.0119, -0.5721],
         [-0.2922, -0.4151,  0.1254, -0.3894],
         [ 0.1214, -0.1452,  0.6938,  0.3128],
         [-1.6108, -0.9612, -1.3380, -0.4233]],

        [[ 1.2708, -0.2156,  1.3569,  0.7860],
         [ 0.3425, -0.7401,  2.3912, -0.4613],
         [ 1.0004,  0.1406,  1.7101, -0.0403],
         [-0.3338, -0.5486,  0.5543, -0.5502],
         [-0.2787, -0.3531, -0.2445, -0.2258],
         

In [24]:
# pass the existing vector through a trainable linear transformation

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

In [25]:
print(Q.shape, K.shape, V.shape)

torch.Size([5, 8, 4]) torch.Size([5, 8, 4]) torch.Size([5, 8, 8])


In [26]:
attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

Let me try and explain what just happened above, let's take example of a single batch

In [27]:
print(Q[0][:3])
print(K[0][:3])

tensor([[ 1.7130, -0.7751,  1.1649, -0.1630],
        [-0.6965, -0.9022,  0.4943, -0.9418],
        [ 1.3131, -0.2510,  1.5398,  0.0729]], grad_fn=<SliceBackward0>)
tensor([[ 0.1849, -1.6876,  0.4281, -1.5150],
        [ 0.4329,  0.4155,  0.4400,  0.8983],
        [-0.3702, -0.7013,  0.4383, -1.2541]], grad_fn=<SliceBackward0>)



well the traditional explanation is:

when the static embedding for a word is passed through these layers it extract specific feature pertaining to corresponding transformation

let's take an example: "The bank of the River"

Query -> transform the static embedding of bank to something like "I am a Noun needing a definition, need to know what am I a river bank, a financial bank, etc"

Key -> transforms the static embedding of River to something like "I am a nature related word, related to river"

Value -> transforms the embedding to actual meaning info, as their it might not be the first attention layer, if it is a second layer, it won't be the absolute meaning

but this never really sat with me completely, I was not able to understand this fully and it seemed pretty handwavy as to explaining what these layers really do, maybe I understand for Value vector but not for Query and Key vector

need to think and understand this properly and clearly once and for all, 
think, what would happen if no linear layer was present


ok let's say no linear layer was present, than the Q*K.T would just give me the cosine similarity between static embeddings of tokens

let's say the attention matrix is

`A`

here `A[i][j]` would be the cosine similarity between the ith token and jth token  (here when I say token I mean the embedding of token)

now let's see what would `A@V` do, here I am assuming `V` to is just the static emebdding


`A_softmax` -> is softmax function applied on `A` across each column (why column? cause weights are distributed across column see below)

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

here `V[i]` represents the static embedding for the ith word, so `V[i]` is a vector, and ``V`[i]`` is the final vector after attention matrix multiplication

``V`[i]`` we can think of it as change or how the original word gets modified

so when taking the weighted sum it tells us, in direction of which static embedding should the input embedding move to the most and least (depending on the weight), so that it represents that modified vector would represent the meaning of the entire sentence, **now I see the poblem with just using static embedding, the problem is if we just use static embedding to compute cosine similarity it would nudge the input emebedding to this layer in the direction which is most similar to the word which we see in the sentence, which we don't always want**

side note: one more thing I learned is that these query and key vector live in a space which is smaller than the static embedding dimension, so it would help computationaly too

In [28]:
# now each element of this of shape context length x context length
attention_matrix.shape

torch.Size([5, 8, 8])

In [29]:
attention_matrix[0]

tensor([[ 2.3704e+00,  7.8561e-01,  6.2427e-01, -5.4162e-01,  4.2974e-01,
          7.4691e-01,  7.5455e-01,  5.2548e-01],
        [ 3.0322e+00, -1.3048e+00,  2.2884e+00,  5.2545e-01,  7.7121e-01,
         -6.6598e-01, -8.2914e-01,  2.0550e+00],
        [ 1.2152e+00,  1.2070e+00,  2.7330e-01, -3.1999e-01,  1.8626e-01,
          1.0241e+00,  6.3459e-01,  1.6197e-03],
        [ 1.1986e+00,  3.5023e-01,  6.8160e-01, -5.2982e-02,  2.2141e-01,
          3.5007e-01,  3.3947e-01,  2.2892e-01],
        [-5.4523e-01,  4.6110e-01, -1.1341e-01,  1.4522e-01, -1.1993e-01,
          3.5072e-01, -2.9104e-02, -3.0452e-01],
        [ 2.4567e+00, -9.5888e-01,  1.7442e+00,  3.2812e-01,  6.0784e-01,
         -4.8340e-01, -5.3004e-01,  1.5617e+00],
        [ 1.0175e+00, -7.0071e-01,  7.0296e-01,  2.5188e-01,  3.1469e-01,
         -3.6366e-01, -7.3586e-01,  1.1049e+00],
        [ 1.4093e-01,  6.0486e-01, -1.7377e-01, -2.2156e-01, -2.0793e-02,
          4.5027e-01,  4.1372e-01, -2.8031e-01]], grad_fn=<Select

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ...

```

now in self attention when we compute the change required for ith vector we don't want any token above ith token to be deciding how it changes, cause during inference we don't have that information at all so we would need to find a way so that above becomes

```

V`[i] = V[0]*A_softmax[i][0] + V[1]*A_softmax[i][1] + V[2]*A_softmax[i][2] + ... + V[i]*A_softmax[i][i]

```

so to do this we convert the lower triangular part equal infinity so that when we do softmax we get zero

In [30]:
print(x_batch[0])
print(y_batch[0])

tensor([37, 32, 41, 18,  3, 31, 43, 36])
tensor([32, 41, 18,  3, 31, 43, 36, 42])


In [31]:
torch.ones_like(attention_matrix[0])

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [32]:
torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool() # should the diagonal also have no contribution?

tensor([[False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False]])

In [33]:
attention_matrix[0].masked_fill(torch.tril(torch.ones_like(attention_matrix[0]), diagonal=-1).bool(), float('-inf'))

tensor([[ 2.3704e+00,  7.8561e-01,  6.2427e-01, -5.4162e-01,  4.2974e-01,
          7.4691e-01,  7.5455e-01,  5.2548e-01],
        [       -inf, -1.3048e+00,  2.2884e+00,  5.2545e-01,  7.7121e-01,
         -6.6598e-01, -8.2914e-01,  2.0550e+00],
        [       -inf,        -inf,  2.7330e-01, -3.1999e-01,  1.8626e-01,
          1.0241e+00,  6.3459e-01,  1.6197e-03],
        [       -inf,        -inf,        -inf, -5.2982e-02,  2.2141e-01,
          3.5007e-01,  3.3947e-01,  2.2892e-01],
        [       -inf,        -inf,        -inf,        -inf, -1.1993e-01,
          3.5072e-01, -2.9104e-02, -3.0452e-01],
        [       -inf,        -inf,        -inf,        -inf,        -inf,
         -4.8340e-01, -5.3004e-01,  1.5617e+00],
        [       -inf,        -inf,        -inf,        -inf,        -inf,
                -inf, -7.3586e-01,  1.1049e+00],
        [       -inf,        -inf,        -inf,        -inf,        -inf,
                -inf,        -inf, -2.8031e-01]],
       grad_fn=

In [34]:
attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))[0]

tensor([[ 2.3704e+00,  7.8561e-01,  6.2427e-01, -5.4162e-01,  4.2974e-01,
          7.4691e-01,  7.5455e-01,  5.2548e-01],
        [       -inf, -1.3048e+00,  2.2884e+00,  5.2545e-01,  7.7121e-01,
         -6.6598e-01, -8.2914e-01,  2.0550e+00],
        [       -inf,        -inf,  2.7330e-01, -3.1999e-01,  1.8626e-01,
          1.0241e+00,  6.3459e-01,  1.6197e-03],
        [       -inf,        -inf,        -inf, -5.2982e-02,  2.2141e-01,
          3.5007e-01,  3.3947e-01,  2.2892e-01],
        [       -inf,        -inf,        -inf,        -inf, -1.1993e-01,
          3.5072e-01, -2.9104e-02, -3.0452e-01],
        [       -inf,        -inf,        -inf,        -inf,        -inf,
         -4.8340e-01, -5.3004e-01,  1.5617e+00],
        [       -inf,        -inf,        -inf,        -inf,        -inf,
                -inf, -7.3586e-01,  1.1049e+00],
        [       -inf,        -inf,        -inf,        -inf,        -inf,
                -inf,        -inf, -2.8031e-01]], grad_fn=<Select

In [35]:
masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

In [36]:
masked_attention_matrix[0]

tensor([[ 2.3704e+00,  7.8561e-01,  6.2427e-01, -5.4162e-01,  4.2974e-01,
          7.4691e-01,  7.5455e-01,  5.2548e-01],
        [       -inf, -1.3048e+00,  2.2884e+00,  5.2545e-01,  7.7121e-01,
         -6.6598e-01, -8.2914e-01,  2.0550e+00],
        [       -inf,        -inf,  2.7330e-01, -3.1999e-01,  1.8626e-01,
          1.0241e+00,  6.3459e-01,  1.6197e-03],
        [       -inf,        -inf,        -inf, -5.2982e-02,  2.2141e-01,
          3.5007e-01,  3.3947e-01,  2.2892e-01],
        [       -inf,        -inf,        -inf,        -inf, -1.1993e-01,
          3.5072e-01, -2.9104e-02, -3.0452e-01],
        [       -inf,        -inf,        -inf,        -inf,        -inf,
         -4.8340e-01, -5.3004e-01,  1.5617e+00],
        [       -inf,        -inf,        -inf,        -inf,        -inf,
                -inf, -7.3586e-01,  1.1049e+00],
        [       -inf,        -inf,        -inf,        -inf,        -inf,
                -inf,        -inf, -2.8031e-01]], grad_fn=<Select

In [37]:
import torch.nn.functional as F


F.softmax(masked_attention_matrix[0], dim=-2)

tensor([[1.0000, 0.8900, 0.1432, 0.1474, 0.2183, 0.2381, 0.2695, 0.0804],
        [0.0000, 0.1100, 0.7561, 0.4284, 0.3072, 0.0580, 0.0553, 0.3711],
        [0.0000, 0.0000, 0.1008, 0.1840, 0.1712, 0.3141, 0.2390, 0.0476],
        [0.0000, 0.0000, 0.0000, 0.2402, 0.1773, 0.1601, 0.1779, 0.0598],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1260, 0.1602, 0.1231, 0.0351],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0696, 0.0746, 0.2266],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0607, 0.1435],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0359]],
       grad_fn=<SoftmaxBackward0>)

### should the diagonal (i.e the current token) have any contribution towards update of input embedding vector?

if it does then ``V`[0] = V[0]`` and that would mean the updated embedding would become ``E_updated[0] = V[0] + E[0]`` need to think more about it

but according to my understanding `V[0]` is same as the absolute embedding of the token so `V[0] + E[0]` would be pracitically `2*absolute_mebdding[0]` since even `E[0]` was created any info about token after that so just itself, but since we don layer norm `norm(2*absolute_embedding[0]) = absolute_embedding[0]` so even if it scales a lot layer normalization brings down the scale of embedding vector so it fits with rest of embedding vector's scale

In [38]:
batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

In [39]:
batch_attention_matrix[0]

tensor([[1.0000, 0.8900, 0.1432, 0.1474, 0.2183, 0.2381, 0.2695, 0.0804],
        [0.0000, 0.1100, 0.7561, 0.4284, 0.3072, 0.0580, 0.0553, 0.3711],
        [0.0000, 0.0000, 0.1008, 0.1840, 0.1712, 0.3141, 0.2390, 0.0476],
        [0.0000, 0.0000, 0.0000, 0.2402, 0.1773, 0.1601, 0.1779, 0.0598],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1260, 0.1602, 0.1231, 0.0351],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0696, 0.0746, 0.2266],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0607, 0.1435],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0359]],
       grad_fn=<SelectBackward0>)

In [40]:
V[0]

tensor([[-1.0649,  0.2965,  0.3353,  0.3442,  1.1092,  0.1191,  0.2104,  0.2798],
        [ 0.0813, -0.2671,  0.2961, -0.0558,  0.3380, -0.8971, -0.9883, -0.4342],
        [-0.1718,  0.7060, -0.4863,  0.2537,  0.5010,  0.6769,  0.6705,  0.5336],
        [-0.1164,  0.2788, -0.5136,  0.2997,  0.1536,  0.3175, -0.3469,  0.3437],
        [ 0.8347,  0.8366,  0.2486, -0.6598,  0.8427,  0.2746,  0.7598,  0.1848],
        [ 0.2959,  0.0736,  0.7743,  0.3301,  0.7697, -0.6246, -0.1260, -0.4732],
        [-0.4222, -0.7692, -0.7086,  0.0442, -0.8891, -0.4178, -0.6331,  0.0332],
        [-0.7886,  0.2428,  0.2275,  0.7752,  0.7137,  0.2354,  0.0599, -0.2116]],
       grad_fn=<SelectBackward0>)

In [41]:
batch_attention_matrix[0]@V[0]

tensor([[-9.5881e-01,  2.1343e-01,  5.1948e-01,  3.8374e-01,  1.6895e+00,
         -7.1798e-01, -6.5419e-01, -6.0015e-02],
        [-2.1320e-01,  9.3265e-01, -3.8861e-01,  4.2061e-01,  1.0010e+00,
          6.6151e-01,  4.6286e-01,  4.5550e-01],
        [ 5.8646e-02,  1.1646e-01, -1.6208e-02,  1.1896e-01,  2.8625e-01,
         -1.1122e-01, -5.4242e-02, -2.1775e-03],
        [ 4.5146e-02,  1.0472e-01, -6.7815e-02,  6.2096e-02,  1.9398e-01,
         -3.5290e-02, -7.7889e-02,  3.2821e-02],
        [ 7.2979e-02,  3.1056e-02,  7.6146e-02,  2.3623e-03,  1.4509e-01,
         -1.0862e-01, -2.5880e-04, -5.5862e-02],
        [-1.8961e-01,  2.7827e-03,  5.2576e-02,  2.0194e-01,  1.4898e-01,
         -2.1264e-02, -4.2416e-02, -7.8406e-02],
        [-1.3880e-01, -1.1843e-02, -1.0366e-02,  1.1393e-01,  4.8460e-02,
          8.4199e-03, -2.9841e-02, -2.8358e-02],
        [-2.8324e-02,  8.7214e-03,  8.1705e-03,  2.7843e-02,  2.5635e-02,
          8.4551e-03,  2.1499e-03, -7.6014e-03]], grad_fn=<MmBack

In [42]:
V_prime = batch_attention_matrix@V

In [43]:
embedding_unnormalized = V_prime + x_embeddings_total

# Layer normalization

we just normalize across the embedding dimension to make sure this addition does not scale the embedding vector to much across each layer so that it has the scale of the original embedding vector

In [44]:
mean_emebdding = embedding_unnormalized.mean(dim=-1)
mean_emebdding.shape

torch.Size([5, 8])

In [45]:
std_embedding = embedding_unnormalized.std(dim=-1)
std_embedding.shape

torch.Size([5, 8])

In [46]:
embedding_unnormalized.shape

torch.Size([5, 8, 8])

In [47]:
print(mean_emebdding[0][0], std_embedding[0][0])

tensor(-0.1837, grad_fn=<SelectBackward0>) tensor(2.4424, grad_fn=<SelectBackward0>)


In [48]:
(embedding_unnormalized[0][0] - mean_emebdding[0][0])/(std_embedding[0][0] + 0.0001)

tensor([-1.5382,  1.0311, -0.0436, -0.3929,  1.1487,  0.4577, -1.2439,  0.5811],
       grad_fn=<DivBackward0>)

In [49]:
embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast
embedding_normalized[0][0]

tensor([-1.5382,  1.0311, -0.0436, -0.3929,  1.1487,  0.4577, -1.2439,  0.5811],
       grad_fn=<SelectBackward0>)

# Multi head self attention

![](20251130180338.png)

Functionaly what multi headed attention does is, it employs multiple smaller self attention blocks, the input to each is the full embedding vectors, and the output are vectors with a smaller dimension


```

let's say the input is [6, 6, 6]  (here 6 represents the dimension of the embedding vector which goes in, and 3 is the context length)


[6, 6, 6] -> head 1 -> [2, 2, 2]'1 (here I am just using '1 as a label)

[6, 6, 6] -> head 2 -> [2, 2, 2]'2

[6, 6, 6] -> head 3 -> [2, 2, 2]'3


concat([2, 2, 2]'1, [2, 2, 2]'2, [2, 2, 2]'3) -> [6, 6, 6]'

[6, 6, 6]' -> linear layer -> [6, 6, 6]'' --> residual connection --> [6, 6, 6]'' + [6, 6, 6] --> layer norm --> [6, 6, 6]final

```



In [50]:
# writing everything we have done in one block

# ----multi head repetition--------
d_k = d_q = 4
d_v = EMBEDDING_DIMENSION

W_q = nn.Linear(EMBEDDING_DIMENSION, d_q, bias = False)
W_k = nn.Linear(EMBEDDING_DIMENSION, d_k, bias = False)
W_v = nn.Linear(EMBEDDING_DIMENSION, d_v, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension

Q, K, V = W_q(x_embeddings_total), W_k(x_embeddings_total), W_v(x_embeddings_total)

attention_matrix = Q@K.transpose(1, 2) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

V_prime = batch_attention_matrix@V
# ----multi head repetition--------

embedding_unnormalized = V_prime + x_embeddings_total

mean_emebdding = embedding_unnormalized.mean(dim=-1)
std_embedding = embedding_unnormalized.std(dim=-1)

embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast


In [51]:
# let me create a class out of the repeated block

class SingleHead():

    def __init__(self, d_k, d_q, d_v, input_dimension) -> None:
        
        self.W_q = nn.Linear(input_dimension, d_q, bias = False)
        self.W_k = nn.Linear(input_dimension, d_k, bias = False)
        self.W_v = nn.Linear(input_dimension, d_v, bias = False) # for single head attention this needs to be same as embedding dimension cause the resulting vector of attention layer get added to input embedding vector so both need to have same dimension
        pass

    def forward(self, input_embeddings):
        Q, K, V = self.W_q(input_embeddings), self.W_k(input_embeddings), self.W_v(input_embeddings)

        attention_matrix = Q@K.transpose(-2, -1) # transpose the 1st and the 2nd dimension not the  this is equivalent to Q[i]@K[i].transpose where i is each element of the batch

        masked_attention_matrix = attention_matrix.masked_fill(torch.tril(torch.ones_like(attention_matrix), diagonal=-1).bool(), float('-inf'))

        batch_attention_matrix = F.softmax(masked_attention_matrix, dim=-2)

        V_prime = batch_attention_matrix@V

        return V_prime

        
    

In [52]:
x_embeddings_total.shape

torch.Size([5, 8, 8])

In [53]:
# since our embedding is of size 8 I am gonna create 2 heads with output dimensino 4

head1, head2 = SingleHead(2, 2, 4, 8), SingleHead(2, 2, 4, 8)

In [54]:
x_out_head_1 = head1.forward(x_embeddings_total)[0]
print(x_out_head_1)
x_out_head_2 = head2.forward(x_embeddings_total)[0]
print(x_out_head_2)
print(torch.cat([x_out_head_1, x_out_head_2], dim=-1))

tensor([[-2.1261, -0.2282, -1.2444,  3.0368],
        [-1.2586,  0.2254, -0.1599,  1.0811],
        [-0.4291, -0.0634, -0.2370,  0.6522],
        [-0.2611, -0.0621, -0.1319,  0.4166],
        [-0.2233, -0.0537, -0.2604,  0.5017],
        [-0.0229, -0.0710, -0.0054,  0.0470],
        [-0.0390, -0.0610, -0.0837,  0.1720],
        [-0.0183, -0.0117, -0.0407,  0.0816]], grad_fn=<SelectBackward0>)
tensor([[ 1.6983e+00,  1.2035e-01, -9.8819e-01, -8.4020e-02],
        [ 2.1684e-01, -1.4396e-02, -1.0625e-02,  4.6400e-02],
        [ 2.8304e-01, -7.9834e-02,  2.2566e-02,  7.2551e-02],
        [ 3.0516e-01,  1.0382e-02,  7.6974e-02,  2.0451e-01],
        [ 2.2057e-01, -4.7918e-03, -2.8258e-02,  5.1018e-02],
        [ 1.1417e-01,  6.4684e-03, -1.8588e-02, -3.3030e-02],
        [ 8.0999e-02, -6.8739e-02,  1.5684e-03, -2.6550e-02],
        [ 1.8603e-01, -3.8074e-01,  2.6809e-02, -1.0420e-01]],
       grad_fn=<SelectBackward0>)
tensor([[-2.1261e+00, -2.2820e-01, -1.2444e+00,  3.0368e+00,  1.6983e+00,

In [55]:
x_out_embedding_total = torch.cat([head1.forward(x_embeddings_total), head2.forward(x_embeddings_total)], dim= -1)
x_out_embedding_total[0]

tensor([[-2.1261e+00, -2.2820e-01, -1.2444e+00,  3.0368e+00,  1.6983e+00,
          1.2035e-01, -9.8819e-01, -8.4020e-02],
        [-1.2586e+00,  2.2544e-01, -1.5994e-01,  1.0811e+00,  2.1684e-01,
         -1.4396e-02, -1.0625e-02,  4.6400e-02],
        [-4.2908e-01, -6.3419e-02, -2.3699e-01,  6.5215e-01,  2.8304e-01,
         -7.9834e-02,  2.2566e-02,  7.2551e-02],
        [-2.6109e-01, -6.2133e-02, -1.3192e-01,  4.1662e-01,  3.0516e-01,
          1.0382e-02,  7.6974e-02,  2.0451e-01],
        [-2.2326e-01, -5.3707e-02, -2.6035e-01,  5.0170e-01,  2.2057e-01,
         -4.7918e-03, -2.8258e-02,  5.1018e-02],
        [-2.2871e-02, -7.1029e-02, -5.4219e-03,  4.7004e-02,  1.1417e-01,
          6.4684e-03, -1.8588e-02, -3.3030e-02],
        [-3.9038e-02, -6.0993e-02, -8.3670e-02,  1.7202e-01,  8.0999e-02,
         -6.8739e-02,  1.5684e-03, -2.6550e-02],
        [-1.8334e-02, -1.1651e-02, -4.0700e-02,  8.1581e-02,  1.8603e-01,
         -3.8074e-01,  2.6809e-02, -1.0420e-01]], grad_fn=<Select

In [56]:
# add and norm 

embedding_unnormalized = x_out_embedding_total + x_embeddings_total
mean_emebdding = embedding_unnormalized.mean(dim=-1)
std_embedding = embedding_unnormalized.std(dim=-1)
embedding_normalized = (embedding_unnormalized - mean_emebdding.unsqueeze(-1))/(std_embedding.unsqueeze(-1) + 0.0001) # unsqueeze needed to broadcast


Now that I have shown functionaly how multi head attention works we need to understand what it really means

it is really like head 1 decides from 0 to 3rd direction which direction it should move in and head 2 decide from 4th to 7th which direction embedding should move in

why we concat here ? why we not add them up? this I did not understand conceptually when I think of multi heads adding up, then I could argue that each head might have its own interpretation of how the space looks like which would might cause contradictory changes to the embeddings, another way to think of it is that adding all of them up would lead to a explotion in the value for each embedding dimension, tho layer norm should fix that

this still needs to be debated, I will have to think more deeply about what it means to add them up vs concat them

ok so it turns out I may have missed the feed forward step which has to be supposedly after every multi head attention layer

So the flow should be

1. Multi-Head self attention
2. Add residual
3. Layer norm
4. feed forward
5. add residual
6. layer norm

in my above example I missed from 3, so let me add that


In [57]:
linear_layer = nn.Linear(EMBEDDING_DIMENSION, EMBEDDING_DIMENSION)

embedding_mixed = linear_layer(embedding_normalized) # embedding_normalized vector supposedly have all the information in bits and piece 0 to 3rd dim comes from head 1 3rd to 6th from another and so on, 
# the embedding_mixed is supposed to have the mixed info about all these layers
embedding_mixed = embedding_normalized + embedding_mixed # residual connection
mean_embedding_mixed = embedding_mixed.mean(dim=-1)
std_embedding_mixed = embedding_mixed.std(dim=-1)

embedding_mixed_normalized = (embedding_mixed - mean_embedding_mixed.unsqueeze(-1))/(std_embedding_mixed.unsqueeze(-1) + 0.0001)

In [58]:
embedding_mixed_normalized[0]

tensor([[-1.6487,  0.5553, -0.7022,  1.2134,  0.2073,  0.0538, -0.8165,  1.1376],
        [ 1.1231,  0.5110, -0.5102,  1.4214,  0.3470, -1.3192, -1.0074, -0.5657],
        [-1.5132,  1.3624, -1.3055,  0.6035, -0.1918,  0.1319,  0.0600,  0.8527],
        [-0.2497,  0.9570, -1.6786,  1.0128,  0.8205, -1.1954,  0.2646,  0.0689],
        [-0.8110,  0.4993, -0.2067,  1.7696, -1.0713, -0.4231, -0.7619,  1.0053],
        [ 1.6586, -0.8625,  0.1600,  0.7321,  0.2747, -0.6146, -1.5630,  0.2146],
        [ 1.1340,  1.1211, -0.2284, -0.7479,  0.0900,  0.1752,  0.3712, -1.9152],
        [-1.6418, -0.4022,  0.8806,  0.8108,  0.9726, -0.4266, -0.9804,  0.7871]],
       grad_fn=<SelectBackward0>)

Ok now this constitutes of the full multihead attention block, I still feel like the explanation:
"Each head computes its own bits of feature that it understands, and then the feed forward layer after that combines and picks and mixes the relevant info to form a refined vector" still a bit hand wavy I might need to actually see what this layer is doing, for that I will have to analyze a already existing pretrained model, which would be the next thing I will do, since I don't have money to buy GPUs I will have to rely on analyzing already trained models, let's continue with next stuff

In [59]:
# project the existing embedding vectors into existing vocab space

project_to_vocab_layer = nn.Linear(EMBEDDING_DIMENSION, len(vocabulary_list))

logits = project_to_vocab_layer(embedding_mixed_normalized)

In [60]:
logits[0].shape # context length, vocab size

torch.Size([8, 44])

In [61]:
# exploding logits and expected value for easy cross entropy computation

logits_exploded = logits.view(logits.shape[0]*logits.shape[1], logits.shape[2])


In [62]:
targets_exploded = y_batch.view(y_batch.shape[0]*y_batch.shape[1])

In [63]:
F.cross_entropy(logits_exploded,targets_exploded)

tensor(3.9654, grad_fn=<NllLossBackward0>)

In [64]:
print(logits_exploded.shape, targets_exploded.shape)

torch.Size([40, 44]) torch.Size([40])


finally completed one single pass through transformers layers, next I will create modules out of it